In [1]:
import feedparser
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point
import fiona
import folium
import os
%matplotlib notebook
from sqlalchemy import create_engine
import psycopg2 
from io import BytesIO as StringIO
from datetime import datetime, timedelta


/Users/michael/GEO/adampy2/lib/python2.7/site-packages/folium/__init__.py:59: UserWarning: This version of folium is the last to support Python 2. Transition to Python 3 to be able to receive updates and fixes. Check out https://python3statement.org/ for more info.
  UserWarning


In [2]:
#Get RSS
feed = 'https://emergency.copernicus.eu/mapping/list-of-components/EMSR348/feed'
gfds = 'http://www.gdacs.org/flooddetection/data.aspx?from=20190323&to=20190323&type=rss&alertlevel=red&datatype=1DAYS'
EMS = 'http://emergency.copernicus.eu/mapping/activations-rapid/feed' #Filter flood #filter disastertype (flood)
gdacs = 'http://www.gdacs.org/xml/rss.xml'
eo = 'https://earthobservatory.nasa.gov/feeds/earth-observatory.rss'
ptc = 'https://ptwc.weather.gov/feeds/ptwc_rss_pacific.xml'

#add more
dlr = 'https://activations.zki.dlr.de/iwg_sem.xml'
icsmd = 'https://disasterscharter.org/charter-portlets/cpi-mvc/activations/feed/rss/'
sertit = 'http://sertit.u-strasbg.fr/RMS/georss/ActivationsSertit.xml'


## GDACS

In [3]:
GDACS_RSS = 'http://www.gdacs.org/xml/rss.xml'

GDACS_URL = [GDACS_RSS]

gdacs_feeds = []
gdacs_iso3 = []
gdacs_country = []
gdacs_etype = []
gdacs_epid = []
gdacs_evid = []
gdacs_guid = []
gdacs_pubdate = []
gdacs_bbox = []
gdacs_coords = []
gdacs_alevel = []
gdacs_ascore = []

wfpiso3 = ['TUR','CHL', 'IDN', 'PHL', 'SDN', 'SSD', 'DRC', 'BGD', 'CAF', 
           'BWA', 'TCD', 'KHM', 'IND', 'MYS','ZWE','MDG','MOZ']

def get_gdacs():
    for url in GDACS_URL:
        gdacs_feeds.append(feedparser.parse(url))

    for feed in gdacs_feeds:
        for post in feed.entries:
            gdacs_iso3.append(post.gdacs_iso3)
            gdacs_country.append(post.gdacs_country)
            gdacs_etype.append(post.gdacs_eventtype)
            gdacs_epid.append(post.gdacs_episodeid)
            gdacs_evid.append(post.gdacs_eventid)
            gdacs_guid.append(post.guid)
            gdacs_pubdate.append(post.published)
            gdacs_bbox.append(post.gdacs_bbox)
            gdacs_coords.append(post.where)
            gdacs_alevel.append(post.gdacs_episodealertlevel)
            gdacs_ascore.append(post.gdacs_alertscore)
            
        guiddf = pd.DataFrame(gdacs_guid)       #0
        pubdatedf = pd.DataFrame(gdacs_pubdate) #1
        isodf = pd.DataFrame(gdacs_iso3)        #2   
        countrydf = pd.DataFrame(gdacs_country) #3
        etypedf = pd.DataFrame(gdacs_etype)     #4
        epidf = pd.DataFrame(gdacs_epid)        #5
        evidf = pd.DataFrame(gdacs_evid)        #6
        bboxdf = pd.DataFrame(gdacs_bbox)       #7
        coordf = pd.DataFrame(gdacs_coords)     #8 #9
        aleveldf = pd.DataFrame(gdacs_alevel)   #10
        ascoredf = pd.DataFrame(gdacs_ascore)   #11
        
        frames = [guiddf, pubdatedf, isodf, countrydf, etypedf, epidf, evidf, bboxdf,coordf, aleveldf, ascoredf]
        df = pd.concat(frames, axis=1, ignore_index = True, names=[frames])
        #return df[df[2].isin(wfpiso3)] #!IMPORTANT!#
        return df



## RELIEF WEB Int

In [4]:
RF_RSS ='https://reliefweb.int/disasters/rss.xml'

RF_URL = [RF_RSS]

rw_feeds = []
rw_pub = []
rw_title = []
rw_category = []
rw_desc = []


def get_rw():
    for url in RF_URL:
        rw_feeds.append(feedparser.parse(url))

    for feed in rw_feeds:
        for post in feed.entries:
            rw_pub.append(post.published)
            rw_title.append(post.title)
            rw_category.append(post.category)
            rw_desc.append(post.description)
            
            
        titledf = pd.DataFrame(rw_title)       #0
        pubdatedf = pd.DataFrame(rw_pub) #1
        catdf = pd.DataFrame(rw_category)        #2   
        descdf = pd.DataFrame(rw_desc)
        
        frames = [titledf,pubdatedf, catdf, descdf]
        df = pd.concat(frames, axis=1, ignore_index = True, names=[frames])
        #return df[df[2].isin(wfpiso3)] #!IMPORTANT!#
        return df

## Copernicus

In [5]:
EMS_RSS = 'http://emergency.copernicus.eu/mapping/activations-rapid/feed'

EMS_URL = [EMS_RSS]

ems_guid = []
ems_title = []
ems_cat = []
ems_geo = []
ems_pub = []
ems_efeeds = []

wfpiso3 = ['TUR','CHL', 'IDN', 'PHL', 'SDN', 'SSD', 'DRC', 'BGD', 'CAF', 
           'BWA', 'TCD', 'KHM', 'IND', 'MYS','ZWE','MDG','MOZ']

def get_ems():
    for url in EMS_URL:
        ems_efeeds.append(feedparser.parse(url))

    for feed in ems_efeeds:
        for post in feed.entries:
            ems_guid.append(post.guid)
            ems_title.append(post.title)
            ems_cat.append(post.category)
            ems_geo.append(post.where)
            ems_pub.append(post.published)
            
        guidf = pd.DataFrame(ems_guid)   #0
        titledf = pd.DataFrame(ems_title)  #1
        catdf= pd.DataFrame(ems_cat)     #2
        geodf= pd.DataFrame(ems_geo)     #3
        pubdf= pd.DataFrame(ems_pub)     #4
        
        frames = [guidf, titledf, catdf, geodf, pubdf]
        df = pd.concat(frames, axis=1, ignore_index = True, names=[frames])
        #return df[df[2].isin(wfpiso3)] #!IMPORTANT!#
        #return df.where[df['category'] == 'FL']
        return df
    

## Global Flood Detection System

In [6]:
GFDS_RSS = 'http://www.gdacs.org/flooddetection/data.aspx?from=20190323&to=20190323&type=rss&alertlevel=red&datatype=7DAYS'

GFDS_URL = [GFDS_RSS]

gfds_desc = []
gfds_title = []
gfds_country = []
gfds_geo = []
gfds_magnitude = []
gfds_pub = []
gfds_feeds = []

wfpiso3 = ['TUR','CHL', 'IDN', 'PHL', 'SDN', 'SSD', 'DRC', 'BGD', 'CAF', 
           'BWA', 'TCD', 'KHM', 'IND', 'MYS','ZWE','MDG','MOZ']

def get_gfds():
    for url in GFDS_URL:
        gfds_feeds.append(feedparser.parse(url))

    for feed in gfds_feeds:
        for post in feed.entries:
            gfds_desc.append(post.description)
            gfds_title.append(post.title)
            gfds_country.append(post.country)
            gfds_geo.append(post.where)
            gfds_magnitude.append(post.items.published)
            #pub.append(post.record_date)
            
        descdf = pd.DataFrame(gfds_desc)   #0
        titledf = pd.DataFrame(gfds_title)  #1
        countrydf= pd.DataFrame(gfds_country)     #2
        geodf= pd.DataFrame(gfds_geo)     #3
        magdf = pd.DataFrame(gfds_magnitude)
        #pubdf= pd.DataFrame(pub)     #4
        
        frames = [descdf,titledf,countrydf,geodf, magdf]
        df = pd.concat(frames, axis=1, ignore_index = True, names=[frames])
        #return df[df[2].isin(wfpiso3)] #!IMPORTANT!#
        #return df.where[df['category'] == 'FL']
        return df
    

## GET DATA

In [12]:
gdacs_data = get_gdacs()
ems_data = get_ems()
#gfds_data = get_gfds()
#rw_data = get_rw()

#reliefweb_f = rw_data.loc[rw_data[2] == 'Flash Flood']
gdacs_fl = gdacs_data.loc[gdacs_data[4] == 'FL']
gdacs_tc = gdacs_data.loc[gdacs_data[4] == 'TC']
ems_f = ems_data.loc[ems_data[2] == 'Flood']
ems_s = ems_data.loc[ems_data[2] == 'Storm']


,0,1,2,3,4,5,6,7,8,9,10,11
4,TC1000562,"Fri, 10 May 2019 12:00:00 GMT",IDN,"Indonesia, Timor-Leste",TC,6,1000562,115.262143342398 141.262143342398 -22.12126548...,"(128.5, -9.3)",Point,Green,1
22,TC1000562,"Fri, 10 May 2019 12:00:00 GMT",IDN,"Indonesia, Timor-Leste",TC,6,1000562,115.262143342398 141.262143342398 -22.12126548...,"(128.5, -9.3)",Point,Green,1
40,TC1000562,"Fri, 10 May 2019 12:00:00 GMT",IDN,"Indonesia, Timor-Leste",TC,6,1000562,115.262143342398 141.262143342398 -22.12126548...,"(128.5, -9.3)",Point,Green,1
58,TC1000562,"Fri, 10 May 2019 12:00:00 GMT",IDN,"Indonesia, Timor-Leste",TC,6,1000562,115.262143342398 141.262143342398 -22.12126548...,"(128.5, -9.3)",Point,Green,1
76,TC1000562,"Fri, 10 May 2019 12:00:00 GMT",IDN,"Indonesia, Timor-Leste",TC,6,1000562,115.262143342398 141.262143342398 -22.12126548...,"(128.5, -9.3)",Point,Green,1
94,TC1000562,"Fri, 10 May 2019 12:00:00 GMT",IDN,"Indonesia, Timor-Leste",TC,6,1000562,115.262143342398 141.262143342398 -22.12126548...,"(128.5, -9.3)",Point,Green,1


In [14]:
#EMS
geometry = [Point(xy) for xy in zip(ems_f[3])]
emsdf = ems_f.drop([3], axis=1)
crs = {'init': 'epsg:4326'}
ems_gdf = GeoDataFrame(emsdf, crs=crs, geometry=geometry)
ems_gdf

,0,1,2,4,5,geometry
4,EMSR352,[EMSR352] Flood in Iran,Flood,Point,"Tue, 09 Apr 2019 17:21:26 +0200",POINT (48.72481 31.38581)
6,EMSR349,"[EMSR349] Flood and Wind Storm in Manicaland, ...",Flood,Point,"Mon, 18 Mar 2019 01:51:55 +0100",POINT (32.03 -19.87)
13,EMSR342,"[EMSR342] Flood in Townsville, Australia",Flood,Point,"Fri, 01 Feb 2019 06:09:34 +0100",POINT (146.73 -19.32)
17,EMSR337,[EMSR337] Flood in Western Greece,Flood,Point,"Mon, 14 Jan 2019 14:22:16 +0100",POINT (21.67 37.58)
21,EMSR333,"[EMSR333] Flood in Sicily, Italy",Flood,Point,"Sun, 04 Nov 2018 23:16:24 +0100",POINT (13.176 37.841)
22,EMSR332,"[EMSR332] Flood in Veneto, Italy",Flood,Point,"Sat, 03 Nov 2018 00:48:54 +0100",POINT (11.84376 45.71602)
24,EMSR330,"[EMSR330] Flood in Sicily, Italy",Flood,Point,"Thu, 25 Oct 2018 19:12:24 +0200",POINT (14.70709 37.28147)
25,EMSR329,"[EMSR329] Flood in southern Sardinia, Italy",Flood,Point,"Wed, 24 Oct 2018 17:53:44 +0200",POINT (9.01 39.25)
27,EMSR327,"[EMSR327] Flash Flood in Andalusia, Spain",Flood,Point,"Tue, 23 Oct 2018 21:06:06 +0200",POINT (-5.08 37.54)
28,EMSR326,"[EMSR326] Flash Flood in Malaga, Spain",Flood,Point,"Sun, 21 Oct 2018 14:34:06 +0200",POINT (-4.97 36.88)


In [15]:
#GDACS
geometry = [Point(xy) for xy in zip(gdacs_data[8])]
gdacsdf = gdacs_data.drop([8], axis=1)
crs = {'init': 'epsg:4326'}
gdacs_gdf = GeoDataFrame(gdacsdf, crs=crs, geometry=geometry)
gdacs_gdf.head()

,0,1,2,3,4,5,6,7,9,10,11,geometry
0,EQ1179070,"Fri, 10 May 2019 08:44:54 GMT",VUT,Vanuatu Islands,EQ,1259730,1179070,165.0856 173.0856 -23.7507 -15.7507,Point,Green,0.01,POINT (169.0856 -19.7507)
1,EQ1179058,"Fri, 10 May 2019 03:23:33 GMT",,Kermadec Islands Region,EQ,1259708,1179058,-180.764 -172.764 -32.6866 -24.6866,Point,Green,0.01,POINT (-176.764 -28.6866)
2,EQ1179027,"Thu, 09 May 2019 23:48:42 GMT",JPN,Japan,EQ,1259696,1179027,127.8456 135.8456 27.78 35.78,Point,Green,0.01,POINT (131.8456 31.78)
3,EQ1179023,"Thu, 09 May 2019 22:43:24 GMT",JPN,Japan,EQ,1259661,1179023,127.8732 135.8732 27.809 35.809,Point,Green,0.01,POINT (131.8732 31.809)
4,TC1000562,"Fri, 10 May 2019 12:00:00 GMT",IDN,"Indonesia, Timor-Leste",TC,6,1000562,115.262143342398 141.262143342398 -22.12126548...,Point,Green,1,POINT (128.5 -9.300000000000001)


In [16]:
geometry = [Point(xy) for xy in zip(ems_data[3])]
emsdf = ems_data.drop([3], axis=1)
crs = {'init': 'epsg:4326'}
ems_gdf = GeoDataFrame(emsdf, crs=crs, geometry=geometry)
ems_gdf.head()

,0,1,2,4,5,geometry
0,EMSR357,[EMSR357] Tropical Cyclone Fani in Eastern India,Storm,Point,"Thu, 02 May 2019 17:38:44 +0200",POINT (85.08911000000001 20.2725)
1,EMSR355,[EMSR355] Tropical Cyclone Kenneth in Comoros,Storm,Point,"Fri, 26 Apr 2019 17:53:06 +0200",POINT (43.33575 -11.39443)
2,EMSR354,"[EMSR354] Tropical Cyclone Kenneth, in Cabo De...",Storm,Point,"Fri, 26 Apr 2019 16:05:38 +0200",POINT (40.47967 -10.6699)
3,EMSR353,"[EMSR353] Wildfire in Lower Saxony, Germany",Wildfire,Point,"Tue, 23 Apr 2019 14:04:18 +0200",POINT (8.3606 52.7088)
4,EMSR352,[EMSR352] Flood in Iran,Flood,Point,"Tue, 09 Apr 2019 17:21:26 +0200",POINT (48.72481 31.38581)


In [19]:
#aoi = gdacsjson.loc[gdacsjson['alertlevelepisode'] == 'Green']

m = folium.Map(tiles='stamentoner') #cartodbpositron #stamentoner #cartodbdark_matter
#folium.GeoJson(ems_gdf).add_to(m)
folium.GeoJson(gdacs_gdf).add_to(m)
m.fit_bounds(m.get_bounds())
m

In [20]:
root = "http://www.gdacs.org/datareport/resources/"
eventTC = "TC/"
eventDR = "DR/"
eventEQ = "EQ/"
eventID = "1000533"
objID = "/geojson_"
objNum = "_1"
objForm = ".geojson"

tceventtest = root + eventTC + eventID + objID + eventID + objNum + objForm

print(tceventtest)

http://www.gdacs.org/datareport/resources/TC/1000533/geojson_1000533_1.geojson


In [21]:
from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
api = SentinelAPI('mgmanalili.updge', 'Suite#41', 'https://scihub.copernicus.eu/dhus')


In [12]:
#officers = '/home/michael/Desktop/gis_pips.csv'
#gis = pd.read_csv(officers)
#rec = gis[(gis['Regional Bureau'] == 'RBN') | (gis['Regional Bureau'] == 'RBJ') | (gis['Regional Bureau'] == 'RBB')]